In [2]:
import warnings
warnings.filterwarnings('ignore')

# Onset 모델 평가

In [7]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def data_generator(cqt_paths, stft_paths, onset_paths, model):
    for cqt_path, stft_path, onset_path in zip(cqt_paths, stft_paths, onset_paths):
        CQT = np.load(cqt_path)
        STFT = np.load(stft_path)
        ONSET = np.load(onset_path)

        # 정규화
        CQT = CQT / np.max(CQT)
        STFT = STFT / np.max(STFT)

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for cqt, stft, onset in zip(CQT, STFT, ONSET):
            yield ({'cqt_input': cqt, 'stft_input': stft}, onset)  # 한 세그먼트씩 반환

def evaluate(test_cqt, test_stft, test_onset):
    test_cqt_paths, test_stft_paths, test_onset_paths = [], [], []

    for cqt_path, stft_path, onset_path in zip(glob.glob(test_cqt), glob.glob(test_stft), glob.glob(test_onset)):
        test_cqt_paths.append(cqt_path)
        test_stft_paths.append(stft_path)
        test_onset_paths.append(onset_path)

    output_signature = (
        {
            'cqt_input': tf.TensorSpec(shape=(100, 264), dtype=tf.float32),
            'stft_input': tf.TensorSpec(shape=(100, 257), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(100, 88), dtype=tf.int8)
    )

    batch_size = 10

    model = load_model('models/onset_detector_v2.h5')

    test_set = tf.data.Dataset.from_generator(lambda: data_generator(test_cqt_paths, test_stft_paths, test_onset_paths, model), output_signature=output_signature)
    test_set = test_set.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    steps_per_epoch = sum([len(np.load(file)) for file in test_cqt_paths]) // batch_size

    loss, accuracy = model.evaluate(test_set, steps=steps_per_epoch, verbose=2)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')

if __name__ == '__main__':
    evaluate('data/preprocessed/testX/cqt/*.npy', 'data/preprocessed/testX/stft/*.npy', 'data/preprocessed/testONSET/*.npy')

570/570 - 9s - loss: 0.0204 - accuracy: 0.0566 - 9s/epoch - 16ms/step
Test Loss: 0.020369313657283783
Test Accuracy: 0.05661052465438843


# Offset 모델 평가

In [8]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def data_generator(cqt_paths, stft_paths, offset_paths, model):
    for cqt_path, stft_path, offset_path in zip(cqt_paths, stft_paths, offset_paths):
        CQT = np.load(cqt_path)
        STFT = np.load(stft_path)
        OFFSET = np.load(offset_path)

        # 정규화
        CQT = CQT / np.max(CQT)
        STFT = STFT / np.max(STFT)

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for cqt, stft, offset in zip(CQT, STFT, OFFSET):
            yield ({'cqt_input': cqt, 'stft_input': stft}, offset)  # 한 세그먼트씩 반환

def evaluate(test_cqt, test_stft, test_offset):
    test_cqt_paths, test_stft_paths, test_offset_paths = [], [], []

    for cqt_path, stft_path, offset_path in zip(glob.glob(test_cqt), glob.glob(test_stft), glob.glob(test_offset)):
        test_cqt_paths.append(cqt_path)
        test_stft_paths.append(stft_path)
        test_offset_paths.append(offset_path)

    output_signature = (
        {
            'cqt_input': tf.TensorSpec(shape=(100, 264), dtype=tf.float32),
            'stft_input': tf.TensorSpec(shape=(100, 257), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(100, 88), dtype=tf.int8)
    )

    batch_size = 10

    model = load_model('models/offset_detector_v2.h5')

    test_set = tf.data.Dataset.from_generator(lambda: data_generator(test_cqt_paths, test_stft_paths, test_offset_paths, model), output_signature=output_signature)
    test_set = test_set.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    steps_per_epoch = sum([len(np.load(file)) for file in test_cqt_paths]) // batch_size

    loss, accuracy = model.evaluate(test_set, steps=steps_per_epoch, verbose=2)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')

if __name__ == '__main__':
    evaluate('data/preprocessed/testX/cqt/*.npy', 'data/preprocessed/testX/stft/*.npy', 'data/preprocessed/testOFFSET/*.npy')

570/570 - 10s - loss: 0.0726 - accuracy: 0.1546 - 10s/epoch - 17ms/step
Test Loss: 0.07262832671403885
Test Accuracy: 0.1546333283185959
